In [16]:
import sys
sys.path.insert(0,'/Users/august/Coding/topic-modelling')

In [17]:
from sklearn.datasets import fetch_20newsgroups
from topic_modelling.data_loader import DataLoader
from topic_modelling.LDA import LDA
import pandas as pd
from pprint import pprint

categories = ['rec.autos', 'talk.religion.misc',
              'comp.graphics', 'sci.space', 'talk.politics.guns']

num_categories = 4
categories = categories[:num_categories]

newsgroups_train = fetch_20newsgroups(subset='train',
                                      categories=categories, remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', 
                                      categories=categories,  remove=('headers', 'footers', 'quotes'))
data_df = pd.DataFrame(newsgroups_train, columns=["data", "target"])
test_data_df = pd.DataFrame(newsgroups_test, columns=["data", "target"])


In [18]:
dl = DataLoader()
corpus = dl.load_df(data_df)

print("prepared data for LDA")

lda = LDA(corpus["data"])
print("initialized LDA")

Processed data
prepared data for LDA
initialized LDA


In [19]:
lda.fit(num_topics=num_categories)

100%|██████████| 50/50 [01:56<00:00,  2.33s/it]

Final perplexity: 2069.1728627109505


In [20]:
dl_test = DataLoader()
test_corpus = dl_test.load_df(test_data_df)

Processed data


In [21]:
predictions = test_corpus["data"].apply(lda.predict_topic)

In [22]:
test_corpus["prediction"] = predictions.apply(lambda rad: rad[0][0]) # predict topic with highest probability

In [23]:
test_corpus.head()

,data,target,prediction
0,"[origin, owner, seat, origin, poster, take, se...",1,2
1,"[went, thru, spin, chair, test, jsc, phd, char...",2,1
2,"[think, let, bhagwan, bhagwan]",3,4
3,"[curious, refer, koresh, man, upper, case, nev...",3,4
4,"[problem, probe, water, definit, com, rubber, ...",1,2


In [36]:
topics, _ = lda.get_topics(max_words_per_topic=5)
topics

[['space', 'nasa', 'orbit', 'launch', 'satellit'],
 ['car', 'would', 'like', 'get', 'use'],
 ['imag', 'file', 'use', 'graphic', 'edu'],
 ['one', 'peopl', 'god', 'say', 'would']]

In [25]:
categories

['rec.autos', 'talk.religion.misc', 'comp.graphics', 'sci.space']

In [31]:
def remap(i: int):
    i -= 1
    cat = [3, 0, 2, 1, 2]
    return cat[i]

In [32]:
test_corpus["prediction"] = test_corpus["prediction"].apply(remap)

In [33]:
test_corpus = test_corpus[test_corpus['data'].map(lambda d: len(d)) > 0]

In [34]:
import numpy as np
from sklearn.metrics import accuracy_score
print("test_data")
accuracy_score(np.array(test_corpus["target"]), test_corpus["prediction"])

test_data


0.3941818181818182

In [35]:
train_predictions = corpus["data"].apply(lda.predict_topic)
corpus["prediction"] = train_predictions.apply(lambda rad: rad[0][0])
corpus["prediction"] = corpus["prediction"].apply(remap)
corpus = corpus[corpus['data'].map(lambda d: len(d)) > 0]
print("train_data")
accuracy_score(np.array(corpus["target"]), corpus["prediction"])

train_data


0.09588377723970944